# Fantasy Points - Calculation
---

### 1. Libraries Import

In [1]:
import warnings
import pandas as pd
from pathlib import Path
import json
import requests

In [2]:
# Suppress the specific warning about data validation
warnings.filterwarnings("ignore", category=UserWarning, message="Data Validation extension is not supported")

# Load the data from the "Fantasy" sheet of the Excel file
excel_path = Path("resources/HMFC_2023-1.xlsx")
sheet_name = "Fantasy"
df = pd.read_excel(excel_path, sheet_name, index_col="Player Full Name")
df.fillna(0, inplace=True)
df.index = df.index.str.replace(',', '')

In [3]:
df.head()

,Matchday Key,Position,Result,Jersey,Hodler Mark Points,Minutes Played,Goal,Goal Assist,Yellow Card,Red Card,Goal Against,Penalties Won,Penalties Converted,Penalties Committed,Penalties Saved,Penalties Missed,Free-kick Converted
Player Full Name,,,,,,,,,,,,,,,,,
Juani Ayerra,PFL Cup-Spring 2023-First Round-Hodler Z,FWD,Win,20,2,25,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
Angelo Zegarra,PFL Cup-Spring 2023-First Round-Hodler Z,GK,Win,1,5,90,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
Julio Villamar,PFL Cup-Spring 2023-First Round-Hodler Z,DEF,Win,2,3,45,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
Carlos Lezama,PFL Cup-Spring 2023-First Round-Hodler Z,DEF,Win,3,3,70,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
Santiago Gil,PFL Cup-Spring 2023-First Round-Hodler Z,DEF,Win,4,3,90,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
def calculate_fantasy_points(row):
    points = 0

    # Minutes Played
    if row["Minutes Played"] < 60:
        points += 1
    else:
        points += 2

    # Goal points based on Position
    if not pd.isna(row["Goal"]):
        if row["Position"] == "GOA" or row["Position"] == "DEF":
            points += 6
        elif row["Position"] == "MID":
            points += 5
        elif row["Position"] == "STR":
            points += 4

    # Goal Assist
    if not pd.isna(row["Goal Assist"]):
        points += 3

    # Clean Sheet points
    if row["Position"] == "GOA" and row["Minutes Played"] > 60:
        points += 4
    elif row["Position"] == "DEF" and row["Minutes Played"] > 60:
        points += 3
    elif row["Position"] == "MID" and row["Minutes Played"] > 60:
        points += 2
    elif row["Position"] == "STR" and row["Minutes Played"] > 60:
        points += 1

    # Missed Penalty
    if not pd.isna(row["Penalties Missed"]):
        points -= 2

    # Saved Penalty
    if not pd.isna(row["Penalties Saved"]):
        points += 5

    # Provoked Penalty
    if row["Penalties Won"] > 0:
        points += 2

    # Penalty Committed
    if not pd.isna(row["Penalties Committed"]):
        points -= 2

    # Conceded Goals points
    if (row["Position"] == "GOA" or row["Position"] == "DEF") and row["Goal Against"] > 0:
        points -= row["Goal Against"] // 2 * 2
    elif (row["Position"] == "MID" or row["Position"] == "STR") and row["Goal Against"] > 0:
        points -= row["Goal Against"] // 2

    # Yellow Card
    if not pd.isna(row["Yellow Card"]):
        points -= 1

    # Second Yellow Card
    if not pd.isna(row["Red Card"]) and row["Red Card"] == "Second Yellow Card":
        points -= 1

    # Red Card
    if not pd.isna(row["Red Card"]) and row["Red Card"] == "Red Card":
        points -= 3
        
    # Hodler points:
    points += row["Hodler Mark Points"]

    return points

# Apply the function to each row of the DataFrame
df["Fantasy Points"] = df.apply(calculate_fantasy_points, axis=1)

df.head()

,Matchday Key,Position,Result,Jersey,Hodler Mark Points,Minutes Played,Goal,Goal Assist,Yellow Card,Red Card,Goal Against,Penalties Won,Penalties Converted,Penalties Committed,Penalties Saved,Penalties Missed,Free-kick Converted,Fantasy Points
Player Full Name,,,,,,,,,,,,,,,,,,
Juani Ayerra,PFL Cup-Spring 2023-First Round-Hodler Z,FWD,Win,20,2,25,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,6
Angelo Zegarra,PFL Cup-Spring 2023-First Round-Hodler Z,GK,Win,1,5,90,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,10
Julio Villamar,PFL Cup-Spring 2023-First Round-Hodler Z,DEF,Win,2,3,45,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,13
Carlos Lezama,PFL Cup-Spring 2023-First Round-Hodler Z,DEF,Win,3,3,70,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,17
Santiago Gil,PFL Cup-Spring 2023-First Round-Hodler Z,DEF,Win,4,3,90,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,17


In [ ]:
json_data = {}
for player_name, player_data in df.iterrows():
    player_dict = player_data.to_dict()
    json_data[player_name] = player_dict

formatted_json = json.dumps(json_data, indent=4)

file_path = 'hodlerfc.json'
with open(file_path, 'w') as json_file:
    json_file.write(formatted_json)